In [1]:
@file:Repository("*mavenLocal")
clearIvyCache("me.hanslovsky")

In [2]:
%use @file[jep.json]
%use serialization

In [3]:
python("a = 123")
python["a"]

123

In [4]:
python["b"] = 456
python("c = a + b")
python["c"]

579

In [5]:
import java.lang.Class
import java.lang.Thread
import java.util.concurrent.Callable
import java.util.concurrent.ExecutorService
import java.util.concurrent.Executors
import jep.Interpreter
import jep.SharedInterpreter


class ThreadsafeInterpreter(
    private val initPython: () -> Interpreter = { SharedInterpreter() },
    threadName: String = "Python"
): Interpreter {
    private lateinit var _python: Interpreter
    private val python: Interpreter
        get() {
            if (!this::_python.isInitialized) _python = initPython()
            return _python
        }
    private val executor = Executors.newSingleThreadExecutor { Thread(it, threadName).also { it.setDaemon(true) } }
    
    override fun close() {
        val f = exec { python.close() }
        executor.shutdown()
        f.get()
    }
    
    override fun eval(str: String) = exec { python.eval(str) }.get()
    override fun exec(str: String) = exec { python.exec(str) }.get()
    override fun runScript(script: String) = exec { python.runScript(script) }.get()
    override fun getValue(str: String) = exec { python.getValue(str) }.get()
    override fun <T> getValue(str: String, clazz: Class<T>) = exec { python.getValue(str, clazz) }.get()
    override operator fun set(name: String, v: Any?) = exec { python.set(name, v) }.get()
    override operator fun invoke(name: String, vararg args: Any) = exec { python.invoke(name, *args) }.get()
    override operator fun invoke(name: String, kwargs: Map<String, Any>) = exec { python.invoke(name, kwargs) }.get()
    override operator fun invoke(name: String, args: Array<Any>, kwargs: Map<String, Any>) = exec { python.invoke(name, kwargs) }.get()
    operator fun get(str: String) = getValue(str)
    
    private fun <T> exec(task: () -> T) = executor.submit(Callable { task() })
}

In [6]:
ThreadsafeInterpreter().let { it.exec("a=1"); it["a"] }

1

In [7]:
import java.lang.Class
import java.lang.Thread
import java.util.concurrent.Callable
import java.util.concurrent.ExecutorService
import java.util.concurrent.Executors
import jep.Interpreter
import jep.Jep
import jep.SharedInterpreter
import jep.python.PyCallable
import jep.python.PyObject


class ThreadsafeInterpreter3(
    private val initPython: () -> Interpreter = { SharedInterpreter() },
    threadName: String = "Python"
): Interpreter {
    
    private lateinit var python: Interpreter
    private val executor = Executors.newSingleThreadExecutor { r -> Thread(r.wrapped, threadName).also { it.setDaemon(true) } }
    
    override fun close() {
        val f = exec { python.close() }
        executor.shutdown()
        f.get()
    }
    
    override fun eval(str: String) = exec { python.eval(str) }.get()
    override fun exec(str: String) = exec { python.exec(str) }.get()
    override fun runScript(script: String) = exec { python.runScript(script) }.get()
    override fun getValue(str: String) = exec { python.getValue(str) }.get()
    override fun <T> getValue(str: String, clazz: Class<T>) = exec { python.getValue(str, clazz) }.get()
    override operator fun set(name: String, v: Any?) = exec { python.set(name, v) }.get()
    override operator fun invoke(name: String, vararg args: Any) = exec { python.invoke(name, *args) }.get()
    override operator fun invoke(name: String, kwargs: Map<String, Any>) = exec { python.invoke(name, kwargs) }.get()
    override operator fun invoke(name: String, args: Array<Any>, kwargs: Map<String, Any>) = exec { python.invoke(name, kwargs) }.get()
    operator fun get(str: String) = getValue(str)
    
    private fun <T> exec(task: () -> T) = executor.submit(Callable { task() })
    private val Runnable.wrapped: Runnable get() = Runnable {
        python = SharedInterpreter()
        try {
            this.run()
        } finally {
            python.close()
        }
    }
    
    fun <T: PyObject> createPyObject(factory: () -> T) = exec { factory() }.get()
}

In [8]:
%%python
import numpy as np
ones = np.ones((1, 2))

In [9]:
import jep.NDArray
val ones = python["ones"] as NDArray<DoubleArray>

In [10]:
ones.data.joinToString()

1.0, 1.0

In [11]:
ones.dimensions.joinToString()

1, 2

In [12]:
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.Json

@Serializable
class Obj {
    var member1: String = ""
    var member2: List<String> = listOf()
    var member3: List<List<String>> = listOf()

    override fun toString() = Json.encodeToString(this)
}

fun fromJson(json: String): Obj = Json.decodeFromString(json)

println(Obj())
val json = """{"member1": "lol", "member3": [["abc"], ["def", "ghi"]]}"""
println(fromJson(json))
println(fromJson(json)::class)


{}
{"member1":"lol","member3":[["abc"],["def","ghi"]]}
class Line_14$Obj
